In [1]:
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import requests
import plotly.express as px

pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: f'%.{2}f' % x)
pd.set_option('display.max_colwidth', None)

In [2]:
url = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"
response = requests.get(url)
data = response.content

In [3]:
df1 = pd.read_xml(data)
df1.drop(['id', 'terminalName', 'installed', 'locked',
          'installDate', 'removalDate', 'temporary'], axis=1, inplace=True)
df1.head()

,name,lat,long,nbBikes,nbStandardBikes,nbEBikes,nbEmptyDocks,nbDocks
0,"River Street , Clerkenwell",51.53,-0.11,14,13,1,5,19
1,"Phillimore Gardens, Kensington",51.50,-0.20,33,33,0,4,37
2,"Christopher Street, Liverpool Street",51.52,-0.08,4,4,0,28,32
3,"St. Chad's Street, King's Cross",51.53,-0.12,16,16,0,7,23
4,"Sedding Street, Sloane Square",51.49,-0.16,26,25,1,1,27


In [4]:
df2 = pd.read_csv('models\datasets\Winter\8Jan2020-14Jan2020.csv')
df2.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation lat,EndStation long,EndStation nbBikes,EndStation nbStandardBikes,EndStation nbEBikes,EndStation nbEmptyDocks,EndStation nbDocks,StartStation lat,StartStation long,StartStation nbBikes,StartStation nbStandardBikes,StartStation nbEBikes,StartStation nbEmptyDocks,StartStation nbDocks
0,94275117,1320,12244,14/01/2020 08:23,345,"Flood Street, Chelsea",14/01/2020 08:01,689,"Spanish Road, Clapham Junction",51.49,-0.17,7,7,0,26,33,51.46,-0.18,7,7,0,23,30
1,94261301,1200,3918,13/01/2020 14:35,689,"Spanish Road, Clapham Junction",13/01/2020 14:15,689,"Spanish Road, Clapham Junction",51.46,-0.18,7,7,0,23,30,51.46,-0.18,7,7,0,23,30
2,94219333,2220,16912,11/01/2020 14:03,819,"Belvedere Road 2, South Bank",11/01/2020 13:26,689,"Spanish Road, Clapham Junction",51.51,-0.11,20,20,0,18,38,51.46,-0.18,7,7,0,23,30
3,94158442,1860,3207,09/01/2020 07:56,44,"Bruton Street, Mayfair",09/01/2020 07:25,689,"Spanish Road, Clapham Junction",51.51,-0.14,6,6,0,15,21,51.46,-0.18,7,7,0,23,30
4,94185200,1800,9755,10/01/2020 07:35,382,"Farm Street, Mayfair",10/01/2020 07:05,689,"Spanish Road, Clapham Junction",51.51,-0.15,2,1,1,13,15,51.46,-0.18,7,7,0,23,30


In [5]:
df = df2.merge(df1, left_on='EndStation Name', right_on='name')
df.drop(['name'], inplace=True, axis=1)
df.columns.values[9:] = 'EndStation ' + df.columns.values[9:]


df = df.merge(df1, left_on='StartStation Name', right_on='name')
df.drop(['name'], inplace=True, axis=1)
df.columns.values[16:] = 'StartStation ' + df.columns.values[16:]

df.set_index(['Rental Id'], inplace=True)
df.head()

,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation EndStation lat,EndStation EndStation long,EndStation EndStation nbBikes,EndStation EndStation nbStandardBikes,EndStation EndStation nbEBikes,EndStation EndStation nbEmptyDocks,EndStation EndStation nbDocks,StartStation EndStation StartStation lat,StartStation EndStation StartStation long,StartStation EndStation StartStation nbBikes,StartStation EndStation StartStation nbStandardBikes,StartStation EndStation StartStation nbEBikes,StartStation EndStation StartStation nbEmptyDocks,StartStation EndStation StartStation nbDocks,StartStation EndStation lat,StartStation EndStation long,StartStation EndStation nbBikes,StartStation EndStation nbStandardBikes,StartStation EndStation nbEBikes,StartStation EndStation nbEmptyDocks,StartStation EndStation nbDocks,StartStation lat,StartStation long,StartStation nbBikes,StartStation nbStandardBikes,StartStation nbEBikes,StartStation nbEmptyDocks,StartStation nbDocks
Rental Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94275117,1320,12244,14/01/2020 08:23,345,"Flood Street, Chelsea",14/01/2020 08:01,689,"Spanish Road, Clapham Junction",51.49,-0.17,7,7,0,26,33,51.46,-0.18,7,7,0,23,30,51.49,-0.17,6,6,0,27,33,51.46,-0.18,6,6,0,24,30
94261301,1200,3918,13/01/2020 14:35,689,"Spanish Road, Clapham Junction",13/01/2020 14:15,689,"Spanish Road, Clapham Junction",51.46,-0.18,7,7,0,23,30,51.46,-0.18,7,7,0,23,30,51.46,-0.18,6,6,0,24,30,51.46,-0.18,6,6,0,24,30
94219333,2220,16912,11/01/2020 14:03,819,"Belvedere Road 2, South Bank",11/01/2020 13:26,689,"Spanish Road, Clapham Junction",51.51,-0.11,20,20,0,18,38,51.46,-0.18,7,7,0,23,30,51.51,-0.11,22,22,0,16,38,51.46,-0.18,6,6,0,24,30
94158442,1860,3207,09/01/2020 07:56,44,"Bruton Street, Mayfair",09/01/2020 07:25,689,"Spanish Road, Clapham Junction",51.51,-0.14,6,6,0,15,21,51.46,-0.18,7,7,0,23,30,51.51,-0.14,12,12,0,9,21,51.46,-0.18,6,6,0,24,30
94185200,1800,9755,10/01/2020 07:35,382,"Farm Street, Mayfair",10/01/2020 07:05,689,"Spanish Road, Clapham Junction",51.51,-0.15,2,1,1,13,15,51.46,-0.18,7,7,0,23,30,51.51,-0.15,1,1,0,14,15,51.46,-0.18,6,6,0,24,30
